In [ ]:
from fastai.vision.all import *
from skimage import measure
from skimage.transform import rescale, resize
from skimage.util import crop, montage
from skimage.morphology import label, square, dilation, watershed, binary_opening
from skimage.io import imsave


from tqdm import tqdm

from PIL import Image

In [ ]:
df= pd.read_csv('../input/cropped-imges-and-masks/crops_with_ships.csv')
df.head()


### Tratamiento del CSV
Como podemos observar, en el CSV va a aparecer una entrada por barco, por lo tanto, cuando aparece más de un barco en la misma imagen, ésta aparecerá tantas veces en el csv como barcos contenga. Vamos a agrupar todos los barcos de la imagen en una sola entrada, agrupando por ImageId y uniendo encoded pixels con un espacio de separación. Además de ello, para mayor facilidad en el entrenamiento posterior añadiremos un nuevo campo que llamaremos "has_ship" que vale 1 en caso de contener barcos y 0 en caso de no contenerlos. https://blog.softhints.com/python-detect-prevent-typeerror/

In [ ]:
# ref: https://www.kaggle.com/kmader/baseline-u-net-model-part-1
def multi_rle_encode(img):
    labels = label(img)
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated: [start0] [length0] [start1] [length1]... in 1d array
    '''
    # reshape to 1d array
    pixels = img.T.flatten() # Needed to align to RLE direction
    # pads the head & the tail with 0 & converts to ndarray
    pixels = np.concatenate([[0], pixels, [0]])
    # gets all start(0->1) & end(1->0) positions 
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    # transforms end positions to lengths
    runs[1::2] -= runs[::2]
    # converts to the string formated: '[s0] [l0] [s1] [l1]...'
    return ' '.join(str(x) for x in runs)

def rle_decode(mask_rle, label=1, shape=(768,768)):
    '''
    mask_rle: run-length as string formated: [start0] [length0] [start1] [length1]... in 1d array
    shape: (height,width) of array to return 
    Returns numpy array according to the shape, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    # gets starts & lengths 1d arrays 
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    # gets ends 1d array
    ends = starts + lengths
    # creates blank mask image 1d array
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    # sets mark pixles
    for lo, hi in zip(starts, ends):
        img[lo:hi] = label
    # reshape as a 2d mask image
    return img.reshape(shape).T  # Needed to align to RLE direction

def masks_as_image(in_mask_list, shape=(768,768)):
    '''Take the individual ship masks and create a single mask array for all ships
    in_mask_list: pd Series: [idx0] [RLE string0]...
    Returns numpy array as (shape.h, sahpe.w, 1)
    '''
    all_masks = np.zeros(shape, dtype = np.int16)
    # if isinstance(in_mask_list, list):
    for label, mask in enumerate(in_mask_list):
        if isinstance(mask, str):
            all_masks += rle_decode(mask,label+1,shape)
    return all_masks

def image_open(img_path):
    return np.array(Image.open(img_path))

def apply_mask(image,mask):
    imax,jmax=mask.shape
    image_masked=np.copy(image)
    for i in range(imax):
        for j in range(jmax):
            if mask[i,j]==1:
                image_masked[i,j,[0,0]]=255
    return image_masked


In [ ]:
mask0=np.zeros([768,768])
rle_encode(mask0)

### Procesado de imagen
Esta parte recoge todas las funciones que vamos a usar para procesar las imágenes antes y después del entrenamiento.

In [ ]:
#recorta la imagen y devuelve un array con 9 trozos
def img_crop(A):
    a = crop(A, ((0, 512), (0, 512), (0,0)), copy=True)
    b = crop(A, ((0, 512), (256, 256), (0,0)), copy=True)
    c = crop(A, ((0, 512), (512, 0), (0,0)), copy=True)
    d = crop(A, ((256, 256), (0, 512), (0,0)), copy=True)
    e = crop(A, ((256, 256), (256, 256), (0,0)), copy=True)
    f = crop(A, ((256, 256), (512, 0), (0,0)), copy=True)
    g = crop(A, ((512, 0), (0, 512), (0,0)), copy=True)
    h = crop(A, ((512, 0), (256, 256), (0,0)), copy=True)
    i = crop(A, ((512, 0), (512, 0), (0,0)), copy=True)
    return[a,b,c,d,e,f,g,h,i]
def mask_crop(A):
    a = crop(A, ((0, 512), (0, 512)), copy=False)
    b = crop(A, ((0, 512), (256, 256)), copy=False)
    c = crop(A, ((0, 512), (512, 0)), copy=False)
    d = crop(A, ((256, 256), (0, 512)), copy=False)
    e = crop(A, ((256, 256), (256, 256)), copy=False)
    f = crop(A, ((256, 256), (512, 0)), copy=False)
    g = crop(A, ((512, 0), (0, 512)), copy=False)
    h = crop(A, ((512, 0), (256, 256)), copy=False)
    i = crop(A, ((512, 0), (512, 0)), copy=False)
    return[a,b,c,d,e,f,g,h,i]

In [ ]:
#from https://github.com/selimsef/dsb2018_topcoders/blob/master/victor/create_masks.py
def trata_mask(mask):
    labels = label(mask)
    tmp = dilation(labels > 0, square(9))    
    tmp2 = watershed(tmp, labels, mask=tmp, watershed_line=True) > 0
    tmp = tmp ^ tmp2
    tmp = dilation(tmp, square(7))
    msk = (255 * tmp).astype('uint8')
    
    props = measure.regionprops(labels)
    msk0 = 255 * (labels > 0)
    msk0 = msk0.astype('uint8')
    
    msk1 = np.zeros_like(labels, dtype='bool')
    
    max_area = np.max([p.area for p in props])
    
    for y0 in range(labels.shape[0]):
        for x0 in range(labels.shape[1]):
            if not tmp[y0, x0]:
                continue
            if labels[y0, x0] == 0:
                if max_area > 4000:
                    sz = 6
                else:
                    sz = 3
            else:
                sz = 3
                if props[labels[y0, x0] - 1].area < 300:
                    sz = 1
                elif props[labels[y0, x0] - 1].area < 2000:
                    sz = 2
            uniq = np.unique(labels[max(0, y0-sz):min(labels.shape[0], y0+sz+1), max(0, x0-sz):min(labels.shape[1], x0+sz+1)])
            if len(uniq[uniq > 0]) > 1:
                msk1[y0, x0] = True
                msk0[y0, x0] = 0
    
    msk1 = 255 * msk1
    msk1 = msk1.astype('uint8')
    
    msk2 = np.zeros_like(labels, dtype='uint8')
    msk = np.stack((msk0, msk1, msk2))
    msk = np.rollaxis(msk, 0, 3)
    
    return msk

### Clasificador
Primero utilizarémos este modelo para **eliminar todas las imágnes que no contengan barcos posibles** para, posteriormente, utilizar un segundo modelo **entrenado exclusivemente para la segmentación en imagénes que contengan barcos**. 


In [ ]:
def get_x(r): return os.path.join('../input/cropped-imges-and-masks/croppedmasks/crops',r['img_name'])
def get_y(r): return r['has_ships']
dblock= DataBlock(blocks=(ImageBlock,CategoryBlock),get_x=get_x, get_y=get_y)
dls=dblock.dataloaders(df, bs=64)
learn=cnn_learner(dls,resnet34, metrics=accuracy)



In [ ]:
if not os.path.isdir('models'):
    !mkdir models
!cp ../input/resnet34-classifier-over-256-crops/models/Resnet34_256_crops.pth models/Resnet34.pth
#deberías hacer esto para cargar los pesos aprendidos
learn.load('Resnet34')
learn_class_inf=learn

## Segmentacion


In [ ]:
class Dice(Metric):
    "Dice coefficient metric for binary target in segmentation"
    def __init__(self, axis=1): self.axis = axis
    def reset(self): self.inter,self.union = 0,0
    def accumulate(self, learn):
        pred,targ = flatten_check(learn.pred.argmax(dim=self.axis), learn.y)
        pred, targ = TensorBase(pred), TensorBase(targ)
        self.inter += (pred*targ).float().sum().item()
        self.union += (pred+targ).float().sum().item()

    @property
    def value(self): return 2. * self.inter/self.union if self.union > 0 else None

def IoU(input, target):
    """Intersection over Union (IoU) metric."""
    input = input.argmax(dim=1).float()
    target = target.squeeze(1).float()
    
    smooth = 1.
    intersection = (input * target).sum()
    union = (input + target).sum() - intersection
    return (intersection + smooth) / (union + smooth)

In [ ]:
df.drop(df[df['has_ships'] == False].index, inplace=True)
df.head()
fnames=[]
for index, row in tqdm(df.iterrows()):
    fnames.append(Path(os.path.join('../input/cropped-imges-and-masks/croppedmasks/crops',row['img_name'])))
    


In [ ]:
def get_y(r): 
    fname=r.stem
    mascara= image_open(os.path.join('../input/cropped-imges-and-masks/croppedmasks/masks','{0}.tif'.format(fname)))
    barcos=mascara[:,:,0]/255
    bordes=2*(mascara[:,:,1]/255)
    return barcos+bordes
   
dls = SegmentationDataLoaders.from_label_func("", bs=32, fnames = fnames, label_func = get_y)
learn= unet_learner(dls,resnet34,n_out=3, metrics=[Dice()])

In [ ]:
if not os.path.isdir('models'):
    !mkdir models
!cp ../input/unet-seg-cropped-3ch-wo-leak/Unet_seg_cropped_256_bs32_3channels_wo_leakage_150epochs.pth models/Unet_seg.pth
#deberías hacer esto para cargar los pesos aprendidos
learn.load('Unet_seg')
learn_seg_inf=learn

## Fase de test
### Primer modelo clasificador determina donde hay y donde no hay barcos



In [ ]:
print('Fase de Test')

In [ ]:
df = pd.read_csv("../input/airbus-ship-detection/sample_submission_v2.csv")
df.head()

In [ ]:
#0d4694897.jpg 0ddc42ee1.jpg 00e5fb033.jpg 1e40229e8 1b5fd69bc 1e415f44b 4d70abd58
path=Path('../input/airbus-ship-detection')
name='000367c13.jpg'

imagen=image_open(path/'test_v2'/name)
type(imagen)
crops=img_crop(imagen)


In [ ]:

with learn_class_inf.no_bar():    
    plt.figure(figsize=(20,20))
    plt.subplot(331), plt.imshow(crops[0]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[0])[0])
    plt.subplot(332), plt.imshow(crops[1]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[1])[0])
    plt.subplot(333), plt.imshow(crops[2]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[2])[0])
    plt.subplot(334), plt.imshow(crops[3]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[3])[0])
    plt.subplot(335), plt.imshow(crops[4]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[4])[0])
    plt.subplot(336), plt.imshow(crops[5]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[5])[0])
    plt.subplot(337), plt.imshow(crops[6]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[6])[0])
    plt.subplot(338), plt.imshow(crops[7]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[7])[0])
    plt.subplot(339), plt.imshow(crops[8]), plt.axis('off') ,plt.title(learn_class_inf.predict(crops[8])[0])

In [ ]:
#with learn_class_inf.no_bar():
   # index=0;
   # for name in tqdm(df['ImageId']):
   #     df.loc[index,'has_ships']=learn_class_inf.predict(image_open(path/'test_v2'/name))[0]
    #    index=1+index

### Modelo segmentador segmenta los barcos 

In [ ]:
masksin=np.array(learn_seg_inf.predict(imagen)[0])

maskt=1*(masksin==1)
labeled=label(maskt, connectivity=1, background=0)
plt.figure(figsize=(20,15))
plt.subplot(131), plt.imshow(imagen), plt.axis('off') ,plt.title(name)
plt.subplot(132), plt.imshow(masksin), plt.axis('off') ,plt.title('Mascara')
plt.subplot(133), plt.imshow(labeled), plt.axis('off') ,plt.title('Labels')
plt.show()



In [ ]:
test=learn_seg_inf.predict(imagen)

In [ ]:
plt.figure(figsize=(20,15))
plt.imshow(np.array(test[2].permute(1,2,0)[:,:,1]))
np.max(np.array(test[2].permute(1,2,0)[:,:,1]))

In [ ]:
masks=[]
for recorte in crops:
    mask=np.zeros([256,256])
    if learn_class_inf.predict(recorte)[0] == 'True':
        mask=np.array(learn_seg_inf.predict(recorte)[0])
        maskt=1*(mask==1)
    masks.append(mask)
mascara=montage(masks)
                               

In [ ]:
plt.figure(figsize=(20,15))
plt.subplot(131), plt.imshow(apply_mask(imagen,mascara)), plt.axis('off') ,plt.title(name)
plt.subplot(132), plt.imshow(apply_mask(imagen,masksin)), plt.axis('off') ,plt.title('Mascara')
plt.subplot(133), plt.imshow(mascara-masksin), plt.axis('off') ,plt.title('Labels')
plt.show()

In [ ]:
out_pred_rows=[]
with learn_class_inf.no_bar():    
    with learn_seg_inf.no_bar():
        index=0
        for name in tqdm(df['ImageId']):
            #print(name)
            imagen=image_open(path/'test_v2'/name)
            crops=img_crop(imagen)
            masks=[]
            count=0
            for recorte in crops:
                if learn_class_inf.predict(recorte)[0] == 'True':
                    mask=np.array(learn_seg_inf.predict(recorte)[0])
                    maskt=1*(mask==1)
                    masks.append(maskt)
                else:
                    mask=np.zeros([256,256])
                    masks.append(mask)
                    count += 1
            if count == 9: 
                out_pred_rows += [{'ImageId': name, 'EncodedPixels': np.nan}]
            else:
                mascara=montage(masks)
                maskt_open= binary_opening(mascara, np.ones((5,5)))
                labeled=label(maskt_open, connectivity=1, background=0)
                sep_rles=multi_rle_encode(labeled)
                if len(sep_rles)>0:
                    for rle in sep_rles:
                        out_pred_rows += [{'ImageId': name, 'EncodedPixels': rle}]
                else: 
                    out_pred_rows += [{'ImageId': name, 'EncodedPixels': np.nan}]
          

In [ ]:
mascara=image_open('../input/cropped-imges-and-masks/croppedmasks/masks/06c7d177f.tif')
plt.imshow(mascara)

In [ ]:
juntos=[]
for name in tqdm(os.listdir('../input/cropped-imges-and-masks/croppedmasks/masks')):
    mascara=image_open('../input/cropped-imges-and-masks/croppedmasks/masks/'+name)
    if np.sum(mascara[:,:,1]) != 0:
        juntos.append(name)

In [ ]:
submission_df = pd.DataFrame(out_pred_rows)[['ImageId', 'EncodedPixels']]
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df

In [ ]:
rle_0 = submission_df.query('ImageId=="000367c13.jpg"')['EncodedPixels']
img_0 = masks_as_image(rle_0)
name='000367c13.jpg'
imagen=image_open(path/'test_v2'/name)

In [ ]:
from skimage.segmentation import mark_boundaries

In [ ]:
plt.figure(figsize=(20,15))
plt.subplot(121), plt.imshow(img_0), plt.axis('off') ,plt.title(name)
plt.subplot(122), plt.imshow(mark_boundaries(imagen,img_0,mode='thick')), plt.axis('off') ,plt.title('Mascara')

plt.show()